In [21]:
def preprocess(dataset_path, num_mfcc=40, n_fft=2048, hop_length =512, num_segment=10):
    data = {"labels": [], "mfcc":[]}
    sample_rate = 22050
    samples_per_segment = int(sample_rate*30/num_segment)
    
    for label_idx, (dirpath, dirnames, filenames) in enumerate (os.walk(dataset_path)):
        if dirpath == dataset_path:
            continue
        for f in sorted(filenames):
            if not f.endswith('.wav'):
                continue
            file_path = str

gi


In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from keras.layers import Dense,Activation,Dropout
DATA_PATH = "E:\Research\ML - Machine Learning\Music Genre Classification\Final Data\mfcc_40.json"


def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    
    return X, y


def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs

        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[1].set_xlabel("Epoch")
    axs[0].set_title("Accuracy evaluation")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error evaluation")
    
    plt.subplots_adjust(hspace=0.8)
    

    plt.show()


def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.

    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split

    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)
    
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    
    

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    
#     # Convert integer labels to one-hot encoding
    y_train = to_categorical(y_train, num_classes=8)
    y_validation = to_categorical(y_validation, num_classes=8)

    return X_train, X_validation, X_test, y_train, y_validation, y_test


def build_model(input_shape):
    """Generates CNN model

    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D(2, padding='valid'))
    #model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D(2, padding='valid'))
    model.add(Dropout(0.3))
    #model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D(1, padding='valid'))
    model.add(Dropout(0.3))
    #model.add(keras.layers.BatchNormalization())

    # output layer
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(8, activation='softmax'))

    return model


def predict(model, X, y):
    """Predict a single sample using the trained model

    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)

    
    print("Target: {}, Predicted label: {}".format(y, predicted_index))


if __name__ == "__main__":

    # get train, validation, test splits
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    # create network
    input_shape = (X_train.shape[1], X_train.shape[2], 1)
    model = build_model(input_shape)

    # compile model
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.summary()
    
   
    earlystop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=10,min_delta=0.0001)
    modelcheck = ModelCheckpoint('best_model.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
    
    # train model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=400, callbacks=[modelcheck])

    # plot accuracy/error for training and validation
    plot_history(history)
    
    y_test = to_categorical(y_test, num_classes=8)
    
    # evaluate model on test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

    # pick a sample to predict from the test set
    X_to_predict = X_test[100]
    y_to_predict = y_test[100]

    # predict sample
    predict(model, X_to_predict, y_to_predict)
    
    genre_names = [
    "Adhunik", "Folk", "Hiphop", "Indie", 
    "Islamic", "Metal", "Pop", "Rock"
    ]
    
    # Generate predictions for the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test = np.argmax(y_test, axis=1)

    
    # Generate classification report
    print(classification_report(y_test, y_pred_classes))

   # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", xticklabels=genre_names, yticklabels=genre_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()  # Show the plot

    # Save the plot after showing it
    plt.savefig("sample_plot.png")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 38, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 19, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 17, 128)       36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 8, 128)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 31, 8, 128)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 6, 128)        1

Epoch 21/400
304/304 [==============================] - ETA: 0s - loss: 0.1645 - accuracy: 0.7134
Epoch 21: val_accuracy did not improve from 0.72383
304/304 [==============================] - 70s 231ms/step - loss: 0.1645 - accuracy: 0.7134 - val_loss: 0.1620 - val_accuracy: 0.7181
Epoch 22/400
304/304 [==============================] - ETA: 0s - loss: 0.1627 - accuracy: 0.7121
Epoch 22: val_accuracy did not improve from 0.72383
304/304 [==============================] - 68s 223ms/step - loss: 0.1627 - accuracy: 0.7121 - val_loss: 0.1633 - val_accuracy: 0.7143
Epoch 23/400
303/304 [============================>.] - ETA: 0s - loss: 0.1616 - accuracy: 0.7161
Epoch 23: val_accuracy improved from 0.72383 to 0.72630, saving model to best_model.hdf5
304/304 [==============================] - 63s 207ms/step - loss: 0.1617 - accuracy: 0.7160 - val_loss: 0.1592 - val_accuracy: 0.7263
Epoch 24/400
303/304 [============================>.] - ETA: 0s - loss: 0.1587 - accuracy: 0.7257
Epoch 24: val

Epoch 49/400
303/304 [============================>.] - ETA: 0s - loss: 0.1201 - accuracy: 0.8005
Epoch 49: val_accuracy did not improve from 0.77865
304/304 [==============================] - 70s 229ms/step - loss: 0.1201 - accuracy: 0.8006 - val_loss: 0.1377 - val_accuracy: 0.7543
Epoch 50/400
303/304 [============================>.] - ETA: 0s - loss: 0.1190 - accuracy: 0.8011
Epoch 50: val_accuracy did not improve from 0.77865
304/304 [==============================] - 71s 232ms/step - loss: 0.1190 - accuracy: 0.8009 - val_loss: 0.1353 - val_accuracy: 0.7663
Epoch 51/400
303/304 [============================>.] - ETA: 0s - loss: 0.1170 - accuracy: 0.8085
Epoch 51: val_accuracy did not improve from 0.77865
304/304 [==============================] - 69s 225ms/step - loss: 0.1169 - accuracy: 0.8086 - val_loss: 0.1361 - val_accuracy: 0.7700
Epoch 52/400
303/304 [============================>.] - ETA: 0s - loss: 0.1162 - accuracy: 0.8102
Epoch 52: val_accuracy improved from 0.77865 to 0.